# Using Regression with CAPM

## REMAKE THIS, ATOMIZE INTO
- Accessing the Data, 

This code is for making a simple CAPM Regression Model using Dr. French's Historical Data and the TD Ameritrade API's Candle Data

It takes data from: 
- TD Ameritrade and tda-api: https://developer.tdameritrade.com/ & https://tda-api.readthedocs.io/en/latest/
- Dr. Kenneth R. French's Data Library(for risk-free rates and market rates): https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

This notebook also utilizes some prewritten functions to pull data without the big setups required for the tda-api, and can be found in the configs.py file in the functions subfolder

The Configs and CSV Data Files will be saved in the repository /Users/dB/.secret/ under the filename 'tda-api-v6.json' and pathname 'ff-research-data'. Their paths are listed here for easier access while I write in this notebook:

/Users/dB/.secret/tda-api-v6.json

/Users/dB/.secret/ff-research-data/

## Step 1: Import Necessary Packages and Add Configs

In [1]:
# Imports -----------------------------------------------------------------------------
import functions_for_notebooks.configs as c
from tda.client import Client
from tda import auth as a
import pandas as pd
import datetime as dt

In [2]:
# Configs -----------------------------------------------------------------------------
tda_api_json_path = '/Users/dB/.secret/tda-api-v6.json'
tda_api_json = c.get_account_data(tda_api_json_path)

ff_data_path = '/Users/dB/.secret/ff-research-data/F-F_Research_Data_Factors.CSV'
ff_data_weekly_path = '/Users/dB/.secret/ff-research-data/F-F_Research_Data_Factors_weekly.CSV'
ff_data_daily_path = '/Users/dB/.secret/ff-research-data/F-F_Research_Data_Factors_daily.csv'

start_datetime = '01/01/2019' # Collect Data from the Last 5 Years

symbol = 'AMD'
end = dt.datetime.now() # Today's Date
start = end - dt.timedelta(days = 1825) # Subtract 5 Years from Today

## Step 2: Collect Data and Process into DataFrames

To collect the data, we need to read Dr. French's CSVs for data and call the TD Ameritrade API for candle data.

Reading Dr. French's CSVs is a simple process, all we need to do is decide to use daily or weekly data and then call Panda's read_from_csv() function. We will need to translate the dates into datetime objects to match up with the API data [ADD LATER]

Calling the API will pull down candle data for a stock. This will provide us with the open, close, high, low and volume data for the specified timeframe. Since Dr. French's Data only has daily and weekly entries for the data, we should only call tda-api's Client.get_price_history_every_day() or Client.get_price_history_every_week() functions so the dates match. 

I have a function from the dBI project named get_price_history_df() that can call the API once a Client object is created, and we can pass in '1d' and '1w' as the periods parameter for the daily and weekly data.

In [12]:
# Get the Data from the CSV
daily_data = c.read_ff_csvs(ff_data_daily_path)
weekly_data = c.read_ff_csvs(ff_data_weekly_path)

daily_data.index.name = 'datetime'
weekly_data.index.name = 'datetime'

# Print the Data
print('Daily Data:')
print(daily_data.head())

print('\nWeekly Data:')
print(weekly_data.head())

Daily Data:
                     Mkt-RF   SMB   HML     RF
datetime                                      
1926-07-01 06:00:00    0.10 -0.25 -0.27  0.009
1926-07-02 06:00:00    0.45 -0.33 -0.06  0.009
1926-07-06 06:00:00    0.17  0.30 -0.39  0.009
1926-07-07 06:00:00    0.09 -0.58  0.02  0.009
1926-07-08 06:00:00    0.21 -0.38  0.19  0.009

Weekly Data:
                     Mkt-RF   SMB   HML     RF
datetime                                      
1926-07-02 06:00:00    1.60 -0.62 -0.83  0.056
1926-07-10 06:00:00    0.36 -0.88  0.31  0.056
1926-07-17 06:00:00    1.01  0.59 -1.44  0.056
1926-07-24 06:00:00   -2.05  0.10 -0.18  0.056
1926-07-31 06:00:00    3.04 -1.82 -0.90  0.056


In [4]:
# Get our API Parameters for the Candle Data Call
ak = tda_api_json['api_key']
ru = tda_api_json['redirect_uri']
tp = tda_api_json['token_path']

# Create the tda.Client object to call the API
client = c.connect_to_api(ak, ru, tp)

# Call the API for candle data
candles_daily = c.get_candles_as_df(c=client, symbol=symbol, periods='1d', start=start, end=end)
candles_weekly = c.get_candles_as_df(c=client, symbol=symbol, periods='1w', start=start, end=end)

# Printing Candles DataFrames:
print('Daily Candles: ')
print(candles_daily.head())
print('\nWeekly Candles: ')
print(candles_weekly.head())

Daily Candles: 
                      open   high    low  close    volume
datetime                                                 
2019-01-14 06:00:00  19.96  20.62  19.75  20.23  71350248
2019-01-15 06:00:00  20.44  20.68  20.26  20.38  62785800
2019-01-16 06:00:00  20.40  20.54  19.71  19.73  70849266
2019-01-17 06:00:00  19.49  20.51  19.02  20.25  85018421
2019-01-18 06:00:00  20.37  21.05  20.02  20.77  88130950

Weekly Candles: 
                       open   high    low  close     volume
datetime                                                   
2019-01-14 06:00:00  19.960  21.05  19.02  20.77  378134685
2019-01-21 06:00:00  20.480  22.03  19.55  21.93  364313502
2019-01-28 06:00:00  20.315  25.14  19.05  24.51  765719550
2019-02-04 06:00:00  24.430  24.66  22.27  23.05  436607483
2019-02-11 06:00:00  23.050  24.05  22.59  23.68  328803629


# Step 3: Merge the DataFrames

This step involves performing an inner-join on the API Data and French's Data DataFrames using DataFrame.concat

In [17]:
# Concatenate the DataFrames
print(daily_data.head())

daily_df = pd.concat([candles_daily, daily_data], axis=1)
weekly_df = pd.concat([candles_weekly, weekly_data], axis=1)
weekly_df = weekly_df.dropna()

# Print the DataFrames
print('Daily DataFrame:')
print(daily_df.head())
print('\nWeekly DataFrame:')
print(weekly_df)

                     Mkt-RF   SMB   HML     RF
datetime                                      
1926-07-01 06:00:00    0.10 -0.25 -0.27  0.009
1926-07-02 06:00:00    0.45 -0.33 -0.06  0.009
1926-07-06 06:00:00    0.17  0.30 -0.39  0.009
1926-07-07 06:00:00    0.09 -0.58  0.02  0.009
1926-07-08 06:00:00    0.21 -0.38  0.19  0.009
Daily DataFrame:
                     open  high  low  close  volume  Mkt-RF   SMB   HML     RF
datetime                                                                      
1926-07-01 06:00:00   NaN   NaN  NaN    NaN     NaN    0.10 -0.25 -0.27  0.009
1926-07-02 06:00:00   NaN   NaN  NaN    NaN     NaN    0.45 -0.33 -0.06  0.009
1926-07-06 06:00:00   NaN   NaN  NaN    NaN     NaN    0.17  0.30 -0.39  0.009
1926-07-07 06:00:00   NaN   NaN  NaN    NaN     NaN    0.09 -0.58  0.02  0.009
1926-07-08 06:00:00   NaN   NaN  NaN    NaN     NaN    0.21 -0.38  0.19  0.009

Weekly DataFrame:
Empty DataFrame
Columns: [open, high, low, close, volume, Mkt-RF, SMB, HML, RF]
In

# Get help from Profs:
- Data is not recent, and how do I concat these and not get an empty dataframe?